In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
from math import pi
from matplotlib.gridspec import GridSpec
from dateutil.relativedelta import *
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import seaborn as sns
import scipy.stats
import pingouin as pg
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from mpl_toolkits.mplot3d import Axes3D
import sklearn.metrics as me
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.model_selection import cross_val_score, cross_val_predict, RepeatedKFold 
from sklearn.linear_model import LinearRegression, LogisticRegression
from collections import Counter

In [82]:
# COMMON VARIABLES

alg_to_analyze = [
    'vader_sentiment_analysis_score',
    'vader_sentiment_analysis_polarity'
]

tpd_columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]

In [83]:
!python3 '/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/main_generate_json_files.py' --all_tweets True

Starting script...
Removing existing file...
Ending script...


In [84]:
data = pd.read_json(r'/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/alldata.json')
data.head()

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
id_str_twitter,23781146,15148309,927028057763995648,77648005,3429765093,10794662,1034514325644431361,529445600,12050562,62314491,18289937,172880784,18609547,2164986296,1026783544457273345,109604376,1040939562774790144,69734383,119897261
name,Fe Rebelatto,Sommelier de Airbnb,✪ Felippe,Conference Distinguished Specialist,Paula Santana,Maharaja of the Legacies,Jessica,"var nickname = ""Riquettinha"";",Roberta Arcoverde,Gabriela,Thamara Andrade,felipe.ts ⚡,Juan Lopes,Rafael Dantas,iagor com i,T̷o̷m̷o̷y̷o̷ ̷ F̷u̷r̷a̷c̷ã̷o̷,Esdras Xavier,belle-🏴.dev || twitch.tv/bellesamways,Willian Justen
screen_name,rebelatto,Guilh_rm_,FelippeRegazio,NannoKa,psanrosa13,rponte,dev_jessi,Riquettinha,rla4,Gabrielathalita,thamyk,felipefialho_,juanplopes,RaffaelDantass,Iagor51,RafaelMansilha,esdras_xavieer,bellesamways,Willian_justen
location,"Rio de Janeiro, Brasil",somewhere in Brazil,,Ctba-PR Brasil,Brasil,"Ceará, Brazil",São Paulo - Brasil,"Belo Horizonte, Brasil","Sao Paulo, Brazil",Belo Horizonte,"Belo Horizonte, Brazil.",Dark Side of the Moon,"Rio de Janeiro, Brasil","São Paulo, Brasil",Pvh,"Palmas, Tocantins, Brasil","Jaraguá do Sul, Brasil",Curitiba,"Petrópolis, Brasil"
url,None,https://t.co/gX4oRZWHg3,https://t.co/KkaPlVH4Wu,https://t.co/hDni4Fh2Ng,https://t.co/YJQuDtv2YW,https://t.co/3yZ94QNQvs,https://t.co/MtUeSEBICu,https://t.co/QU71LJgSuD,None,https://t.co/AmkOKWlftK,https://t.co/JWBzdmQOcV,https://t.co/yPuEuh1bn9,https://t.co/Uhw9rEUK2v,None,None,https://t.co/9cS34v6hKc,https://t.co/H7Mzn0CEHR,https://t.co/bf6AeBboQk,https://t.co/23lnSqtqOj


In [85]:
data_psi = pd.read_csv(r'/Users/leosilva/Documents/Estudo/Doutorado/Coimbra/2019-2020/Disciplinas/Thesis/analises_psicologos/Tweets_consolidados.csv')
data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3
0,102600,Negativo Fraco,Positivo Fraco,Neutro
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte
2,103085,Negativo Forte,Negativo Forte,Negativo Forte
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte


In [86]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI1'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI1'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI1'] == 'Neutro'),
    (data_psi['Classificacao_PSI1'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI1'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI1_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos


In [87]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI2'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI2'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI2'] == 'Neutro'),
    (data_psi['Classificacao_PSI2'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI2'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI2_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg


In [88]:
# create a list of our conditions
conditions = [
    (data_psi['Classificacao_PSI3'] == 'Positivo Fraco'),
    (data_psi['Classificacao_PSI3'] == 'Positivo Forte'),
    (data_psi['Classificacao_PSI3'] == 'Neutro'),
    (data_psi['Classificacao_PSI3'] == 'Negativo Fraco'),
    (data_psi['Classificacao_PSI3'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['pos', 'pos', 'neu', 'neg', 'neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Classificacao_PSI3_3_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Classificacao_PSI1,Classificacao_PSI2,Classificacao_PSI3,Classificacao_PSI1_3_cat,Classificacao_PSI2_3_cat,Classificacao_PSI3_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos,neu
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg,neg


In [89]:
data_psi.columns = ['ID',
                    'Class_PSI1_5_cat',
                    'Class_PSI2_5_cat',
                    'Class_PSI3_5_cat',
                    'Class_PSI1_3_cat',
                    'Class_PSI2_3_cat',
                    'Class_PSI3_3_cat']

data_psi.head()

,ID,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,Negativo Fraco,Positivo Fraco,Neutro,neg,pos,neu
1,102895,Negativo Fraco,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,Negativo Forte,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,Positivo Fraco,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,Positivo Fraco,Negativo Fraco,Negativo Forte,pos,neg,neg


In [90]:
# create a list of our conditions
conditions = [
    (data_psi['Class_PSI1_5_cat'] == 'Positivo Fraco'),
    (data_psi['Class_PSI1_5_cat'] == 'Positivo Forte'),
    (data_psi['Class_PSI1_5_cat'] == 'Neutro'),
    (data_psi['Class_PSI1_5_cat'] == 'Negativo Fraco'),
    (data_psi['Class_PSI1_5_cat'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Class_PSI1_5_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,w_neg,Positivo Fraco,Neutro,neg,pos,neu
1,102895,w_neg,Negativo Forte,Negativo Forte,neg,neg,neg
2,103085,s_neg,Negativo Forte,Negativo Forte,neg,neg,neg
3,103092,w_pos,Positivo Fraco,Positivo Fraco,pos,pos,pos
4,103200,w_pos,Negativo Fraco,Negativo Forte,pos,neg,neg


In [91]:
# create a list of our conditions
conditions = [
    (data_psi['Class_PSI2_5_cat'] == 'Positivo Fraco'),
    (data_psi['Class_PSI2_5_cat'] == 'Positivo Forte'),
    (data_psi['Class_PSI2_5_cat'] == 'Neutro'),
    (data_psi['Class_PSI2_5_cat'] == 'Negativo Fraco'),
    (data_psi['Class_PSI2_5_cat'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Class_PSI2_5_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,w_neg,w_pos,Neutro,neg,pos,neu
1,102895,w_neg,s_neg,Negativo Forte,neg,neg,neg
2,103085,s_neg,s_neg,Negativo Forte,neg,neg,neg
3,103092,w_pos,w_pos,Positivo Fraco,pos,pos,pos
4,103200,w_pos,w_neg,Negativo Forte,pos,neg,neg


In [92]:
# create a list of our conditions
conditions = [
    (data_psi['Class_PSI3_5_cat'] == 'Positivo Fraco'),
    (data_psi['Class_PSI3_5_cat'] == 'Positivo Forte'),
    (data_psi['Class_PSI3_5_cat'] == 'Neutro'),
    (data_psi['Class_PSI3_5_cat'] == 'Negativo Fraco'),
    (data_psi['Class_PSI3_5_cat'] == 'Negativo Forte')
    ]

# create a list of the values we want to assign for each condition
values = ['w_pos', 's_pos', 'neu', 'w_neg', 's_neg']

# create a new column and use np.select to assign values to it using our lists as arguments
data_psi['Class_PSI3_5_cat'] = np.select(conditions, values)

data_psi.head()

,ID,Class_PSI1_5_cat,Class_PSI2_5_cat,Class_PSI3_5_cat,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat
0,102600,w_neg,w_pos,neu,neg,pos,neu
1,102895,w_neg,s_neg,s_neg,neg,neg,neg
2,103085,s_neg,s_neg,s_neg,neg,neg,neg
3,103092,w_pos,w_pos,w_pos,pos,pos,pos
4,103200,w_pos,w_neg,s_neg,pos,neg,neg


In [93]:
tweets = []

for d in data:
    if data[d]['bigfive']:
        tweets.append(data[d]['tweets'])

final_tweets = []
for i in tweets:
    final_tweets.extend(i)

In [265]:
print(len(final_tweets))

91632


In [196]:
def clean_df(df, algs):
    clean_df_columns = [
        'text',
        'created_at',
        'id_user',
        'final_score',
        'final_polarity',
        'vader_sentiment_analysis_score',
        'vader_sentiment_analysis_polarity',
        'oplexicon_sentiment_analysis_score',
        'oplexicon_sentiment_analysis_polarity',
        'sentistrength_sentiment_analysis_score',
        'sentistrength_sentiment_analysis_polarity',
        'sentilexpt_sentiment_analysis_score',
        'sentilexpt_sentiment_analysis_polarity'
    ]
    for alg in algs:
        if alg == 'vader':
            clean_df_columns.remove('vader_sentiment_analysis_score')
            clean_df_columns.remove('vader_sentiment_analysis_polarity')
        elif alg == 'oplexicon':
            clean_df_columns.remove('oplexicon_sentiment_analysis_score')
            clean_df_columns.remove('oplexicon_sentiment_analysis_polarity')
        elif alg == 'sentistrength':
            clean_df_columns.remove('sentistrength_sentiment_analysis_score')
            clean_df_columns.remove('sentistrength_sentiment_analysis_polarity')
        elif alg == 'sentilexpt':
            clean_df_columns.remove('sentilexpt_sentiment_analysis_score')
            clean_df_columns.remove('sentilexpt_sentiment_analysis_polarity')
        
    for c in clean_df_columns:
        del df[c]

In [197]:
def create_tweets_pd(data, id_user):

    tweets_pd = pd.DataFrame(data[id_user]['tweets'])
    del tweets_pd[0]
    del tweets_pd[1]
    del tweets_pd[4]
    del tweets_pd[5]
    del tweets_pd[6]
    del tweets_pd[18]
    del tweets_pd[19]
    del tweets_pd[20]
    
    tweets_pd.columns = ["text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "final_score", "final_polarity"]
 
    return tweets_pd

In [198]:
def create_tweets_df(tweets):
    df = pd.DataFrame(final_tweets)

    del df[1]
    del df[4]
    del df[5]
    del df[6]
    del df[18]
    del df[19]
    del df[20]

    df.columns = ["id", "text", "created_at", "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                             "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                             "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                             "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                             "final_score", "final_polarity"]
    return df

In [231]:
def filter_tweets_by_manually_analyzed(df, data_psi, algs):
    filtered_list = []
    for d in df['id']:
        if not (data_psi.loc[data_psi['ID'] == d]).empty:
            t_psi = data_psi.loc[data_psi['ID'] == d]
            t_o = df.loc[df['id'] == d]
            if len(algs) > 1:
                scores = []
                mean = 0
                pol = ''
                for a in algs:
                    scores.append(t_o["{}_sentiment_analysis_score".format(a)].values[0])
                    mean = np.mean(scores)
                    if mean > 0.0:
                        pol = 'pos'
                    elif mean < 0.0:
                        pol = 'neg'
                    else:
                        pol = 'neu'
                dic = {
                    'id': d,
                    'analysis_score': mean,
                    'analysis_polarity': pol,
                    'Class_PSI1_3_cat': t_psi['Class_PSI1_3_cat'].values[0],
                    'Class_PSI2_3_cat': t_psi['Class_PSI2_3_cat'].values[0],
                    'Class_PSI3_3_cat': t_psi['Class_PSI3_3_cat'].values[0]
                }
                filtered_list.append(dic)
            else:
                dic = {
                    'id': d,
                    'analysis_score': t_o[t_o.keys()[1]].values[0],
                    'analysis_polarity': t_o[t_o.keys()[2]].values[0],
                    'Class_PSI1_3_cat': t_psi['Class_PSI1_3_cat'].values[0],
                    'Class_PSI2_3_cat': t_psi['Class_PSI2_3_cat'].values[0],
                    'Class_PSI3_3_cat': t_psi['Class_PSI3_3_cat'].values[0]
                }
                filtered_list.append(dic)
    
    filtered_df = pd.DataFrame(filtered_list)
    return filtered_df

In [232]:
def fill_manual_polarities():
    for d in df_n['id']:
        pol = []
        t = df_n.query('id == {}'.format(d))
        pol.append(t['Class_PSI1_3_cat'].values[0])
        pol.append(t['Class_PSI2_3_cat'].values[0])
        pol.append(t['Class_PSI3_3_cat'].values[0])
        c = dict(Counter(pol))
        for i in c.items():
            if i[1] >= 2:
                df_n.loc[df_n['id'] == d, 'manual_polarity'] = i[0]

In [233]:
def conf_matrix(x, col1, col2):
    if x[col1] == 1 and x[col2] == 1:
        return 'TP'
    elif x[col1] == 1 and x[col2] == -1:
        return 'FN'
    elif x[col1] == -1 and x[col2] == 1:
        return 'FP'
    elif x[col1] == -1 and x[col2] == -1:
        return 'TN'
    else:
        return 0

In [241]:
def perform_statistics():
    df_n['manual_prediction'] = df_n['manual_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))
    df_n['analysis_prediction'] = df_n['analysis_polarity'].apply(lambda x: -1 if x == 'neg' else (1 if x == 'pos' else 0))

    df_n['accuracy'] = df_n.apply(lambda x: 1 if x['manual_prediction'] == x['analysis_prediction'] else 0, axis=1)

    df_n['conf_matrix'] = df_n.apply(lambda x: conf_matrix(x, 'manual_prediction', 'analysis_prediction'), axis=1)

    list_precision = []

    conf_vals = df_n['conf_matrix'].value_counts().to_dict()

    accuracy = (conf_vals['TP'] + conf_vals['TN']) / (conf_vals['TP'] + conf_vals['TN'] + conf_vals['FP'] + conf_vals['FN'])
    precision_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FP'])

    # ------------------------------------------------------
    # verificar se devo incluir os NEUTROS também nas contas
    # ------------------------------------------------------

    recall_pos = conf_vals['TP'] / (conf_vals['TP'] + conf_vals['FN'])
    f1_score_pos = 2*precision_pos*recall_pos / (precision_pos + recall_pos)

    precision_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FN'])
    recall_neg = conf_vals['TN'] / (conf_vals['TN'] + conf_vals['FP'])
    f1_score_neg = 2*precision_neg*recall_neg / (precision_neg + recall_neg)
    
    lexicon = alg[0] if len(alg) == 1 else " + ".join(alg)

    d = {
        'Sentiment Lexicon': lexicon,
        'Accuracy': accuracy,
        'Precision - Positive': precision_pos,
        'Recall - Positive': recall_pos,
        'F1-Score - Positive': f1_score_pos,
        'Precision - Negative': precision_neg,
        'Recall - Negative': recall_neg,
        'F1-Score - Negative': f1_score_neg
    }
    
    return d

In [256]:
rdf = pd.DataFrame(columns=['Sentiment Lexicon', 'Accuracy', 'Precision - Positive', 
                                'Recall - Positive', 'F1-Score - Positive',
                                'Precision - Negative', 
                                'Recall - Negative', 'F1-Score - Negative'
                               ])

In [236]:
# --------------
# MANUAL X VADER
# --------------

alg = ['vader']

df = create_tweets_df(final_tweets)
clean_df(df, alg)
df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
fill_manual_polarities()
d = perform_statistics()
rdf = rdf.append(d, ignore_index=True)

In [237]:
# ------------------
# MANUAL X OPLEXICON
# ------------------

alg = ['oplexicon']

df = create_tweets_df(final_tweets)
clean_df(df, alg)
df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
fill_manual_polarities()
d = perform_statistics()
rdf = rdf.append(d, ignore_index=True)

In [238]:
# ----------------------
# MANUAL X SENTISTRENGTH
# ----------------------

alg = ['sentistrength']

df = create_tweets_df(final_tweets)
clean_df(df, alg)
df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
fill_manual_polarities()
d = perform_statistics()
rdf = rdf.append(d, ignore_index=True)

In [239]:
# --------------------
# MANUAL X SENTILEX-PT
# --------------------

alg = ['sentilexpt']

df = create_tweets_df(final_tweets)
clean_df(df, alg)
df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
fill_manual_polarities()
d = perform_statistics()
rdf = rdf.append(d, ignore_index=True)

In [242]:
# -------------------------------------
# MANUAL X ENSAMBLE (VADER + OPLEXICON)
# -------------------------------------

alg = ['vader', 'oplexicon']

df = create_tweets_df(final_tweets)
clean_df(df, alg)
df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
fill_manual_polarities()
d = perform_statistics()
rdf = rdf.append(d, ignore_index=True)

In [244]:
# -----------------------------------------
# MANUAL X ENSAMBLE (VADER + SENTISTRENGTH)
# -----------------------------------------

alg = ['vader', 'sentistrength']

df = create_tweets_df(final_tweets)
clean_df(df, alg)
df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
fill_manual_polarities()
d = perform_statistics()
rdf = rdf.append(d, ignore_index=True)

In [245]:
# -----------------------------------------
# MANUAL X ENSAMBLE (VADER + SENTILEX-PT)
# -----------------------------------------

alg = ['vader', 'sentilexpt']

df = create_tweets_df(final_tweets)
clean_df(df, alg)
df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
fill_manual_polarities()
d = perform_statistics()
rdf = rdf.append(d, ignore_index=True)

In [246]:
rdf

,Sentiment Lexicon,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative
0,vader,0.662222,0.713178,0.702290,0.707692,0.593750,0.606383,0.600000
1,oplexicon,0.673684,0.700730,0.820513,0.755906,0.603774,0.438356,0.507937
2,sentistrength,0.768293,0.807018,0.851852,0.828829,0.680000,0.607143,0.641509
3,sentilexpt,0.724551,0.736842,0.769231,0.752688,0.708333,0.671053,0.689189
4,vader + oplexicon,0.678161,0.701754,0.784314,0.740741,0.633333,0.527778,0.575758
5,vader + sentistrength,0.691057,0.738562,0.758389,0.748344,0.612903,0.587629,0.600000
6,vader + sentilexpt,0.682540,0.721088,0.731034,0.726027,0.628571,0.616822,0.622642


In [258]:
import copy

combs = [] # lista que guarda a combinacao de todos os algoritmos

def combinations(target,data):
    for i in range(len(data)):
        new_target = copy.copy(target)
        new_data = copy.copy(data)
        new_target.append(data[i])
        new_data = data[i+1:]
        combs.append(new_target)
        combinations(new_target, new_data)

target = []
alg = ['vader', 'oplexicon', 'sentistrength', 'sentilexpt']
combinations(target, alg)

print(combs)

[['vader'], ['vader', 'oplexicon'], ['vader', 'oplexicon', 'sentistrength'], ['vader', 'oplexicon', 'sentistrength', 'sentilexpt'], ['vader', 'oplexicon', 'sentilexpt'], ['vader', 'sentistrength'], ['vader', 'sentistrength', 'sentilexpt'], ['vader', 'sentilexpt'], ['oplexicon'], ['oplexicon', 'sentistrength'], ['oplexicon', 'sentistrength', 'sentilexpt'], ['oplexicon', 'sentilexpt'], ['sentistrength'], ['sentistrength', 'sentilexpt'], ['sentilexpt']]


In [259]:
for alg in combs:
    df = create_tweets_df(final_tweets)
    clean_df(df, alg)
    df_n = filter_tweets_by_manually_analyzed(df, data_psi, alg)
    fill_manual_polarities()
    d = perform_statistics()
    rdf = rdf.append(d, ignore_index=True)

In [260]:
rdf

,Sentiment Lexicon,Accuracy,Precision - Positive,Recall - Positive,F1-Score - Positive,Precision - Negative,Recall - Negative,F1-Score - Negative
0,vader,0.662222,0.713178,0.702290,0.707692,0.593750,0.606383,0.600000
1,vader + oplexicon,0.678161,0.701754,0.784314,0.740741,0.633333,0.527778,0.575758
2,vader + oplexicon + sentistrength,0.697761,0.721591,0.798742,0.758209,0.652174,0.550459,0.597015
3,vader + oplexicon + sentistrength + sentilexpt,0.703422,0.735294,0.791139,0.762195,0.645161,0.571429,0.606061
4,vader + oplexicon + sentilexpt,0.702703,0.733333,0.785714,0.758621,0.648936,0.580952,0.613065
5,vader + sentistrength,0.691057,0.738562,0.758389,0.748344,0.612903,0.587629,0.600000
6,vader + sentistrength + sentilexpt,0.706564,0.737500,0.776316,0.756410,0.656566,0.607477,0.631068
7,vader + sentilexpt,0.682540,0.721088,0.731034,0.726027,0.628571,0.616822,0.622642
8,oplexicon,0.673684,0.700730,0.820513,0.755906,0.603774,0.438356,0.507937
9,oplexicon + sentistrength,0.722222,0.738562,0.849624,0.790210,0.682540,0.518072,0.589041


In [264]:
df_n

,id,analysis_score,analysis_polarity,Class_PSI1_3_cat,Class_PSI2_3_cat,Class_PSI3_3_cat,manual_polarity,manual_prediction,analysis_prediction,accuracy,conf_matrix
0,102600,0.156416,pos,neg,pos,neu,NaN,0,1,0,0
1,102895,0.250000,pos,neg,neg,neg,neg,-1,1,0,FP
2,103085,-0.718421,neg,neg,neg,neg,neg,-1,-1,1,TN
3,103092,0.250000,pos,pos,pos,pos,pos,1,1,1,TP
4,103200,0.000000,neu,pos,neg,neg,neg,-1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
450,194153,0.000000,neu,neg,neu,neu,neu,0,0,1,0
451,194416,-0.250000,neg,neg,neg,neu,neg,-1,-1,1,TN
452,194647,0.000000,neu,neg,neg,pos,neg,-1,0,0,0
453,194664,0.000000,neu,neu,pos,neg,NaN,0,0,1,0


In [263]:
df_n['manual_polarity'].isna().sum()

47

In [ ]:
def create_data():
    bar_chart_data = []

    count = 1
    for i in data:
        if data[i]['bigfive']:
            tweets = create_tweets_pd(data, i)

            d = ['P{}'.format(count),
                 len(tweets.query("final_polarity == 'pos'")),
                 len(tweets.query("final_polarity == 'neu'")), 
                 len(tweets.query("final_polarity == 'neg'")),
                 (len(tweets.query("final_polarity == 'pos'")) / len(tweets)) * 100,
                 (len(tweets.query("final_polarity == 'neu'")) / len(tweets)) * 100,
                 (len(tweets.query("final_polarity == 'neg'")) / len(tweets)) * 100,
                 data[i]['bigfive'][0][1],
                 data[i]['bigfive'][0][2],
                 data[i]['bigfive'][0][3],
                 data[i]['bigfive'][0][4],
                 data[i]['bigfive'][0][5]
                ]
            bar_chart_data.append(d)
            count += 1

    df = pd.DataFrame(bar_chart_data, columns=[
        "Participant","pos", "neu", "neg", 
        "por_pos","por_neu","por_neg", "O", "C", "E", "A", "N"])
    
    biglist = ['O', 'C', 'E', 'A', 'N']
    for b in biglist:
        res = []
        for r in df[b]:
            if r <= 10:
                res.append(1)
            elif r > 10 and r < 21:
                res.append(2)
            elif r > 20 and r < 31:
                res.append(3)
            elif r > 30 and r < 41:
                res.append(4)
            elif r > 40:
                res.append(5)
        
        df['{}_Coded'.format(b)] = res
    
    return df

In [ ]:
df = create_data()

In [ ]:
df

In [ ]:
type(df['C'])

In [ ]:
print(list(df['A']))

In [ ]:
pg.normality(df)

In [ ]:
pg.homoscedasticity(df)

In [ ]:
pg.anova(dv='pos', between=['O'], data=df).round(3)

In [ ]:
pg.anova(dv='pos', between=['E'], data=df).round(3)

In [ ]:
pg.anova(dv='pos', between=['O', 'A'], data=df).round(3)

In [ ]:
pg.anova(dv='pos', between=['O', 'N'], data=df).round(3)

In [ ]:
stats.f_oneway(df['pos'], df['O'])

In [ ]:
model = ols('pos ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
model = ols('neg ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
model = ols('por_pos ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
model = ols('por_neg ~ O + C + E + A + N', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['O'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['C'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['E'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['N'], alpha=0.05)
print(m_comp)

In [ ]:
model = ols(formula='pos ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
model = ols('neg ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
model = ols('por_pos ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
model = ols('por_neg ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
aov = sm.stats.anova_lm(model, typ=2)
aov

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['pos'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['neg'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_pos'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['O_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['C_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['E_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['A_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
# perform multiple pairwise comparison (Tukey HSD)
m_comp = pairwise_tukeyhsd(endog=df['por_neg'], groups=df['N_Coded'], alpha=0.05)
print(m_comp)

In [ ]:
pg.ttest(df['pos'], df['O'])

In [ ]:
pg.ttest(df['pos'], df['C'])

In [ ]:
pg.ttest(df['pos'], df['E'])

In [ ]:
pg.ttest(df['pos'], df['A'])

In [ ]:
pg.ttest(df['pos'], df['N'])

In [ ]:
print(pg.mwu(df['pos'], df['O']))
print(pg.mwu(df['neg'], df['O']))

In [ ]:
print(pg.mwu(df['pos'], df['C']))
print(pg.mwu(df['neg'], df['C']))

In [ ]:
print(pg.mwu(df['pos'], df['E']))
print(pg.mwu(df['neg'], df['E']))

In [ ]:
print(pg.mwu(df['pos'], df['A']))
print(pg.mwu(df['neg'], df['A']))

In [ ]:
print(pg.mwu(df['pos'], df['N']))
print(pg.mwu(df['neg'], df['N']))

In [ ]:
print(pg.mwu(df['por_pos'], df['O']))
print(pg.mwu(df['por_neg'], df['O']))

In [ ]:
print(pg.mwu(df['por_pos'], df['C']))
print(pg.mwu(df['por_neg'], df['C']))

In [ ]:
print(pg.mwu(df['por_pos'], df['E']))
print(pg.mwu(df['por_neg'], df['E']))

In [ ]:
print(pg.mwu(df['por_pos'], df['A']))
print(pg.mwu(df['por_neg'], df['A']))

In [ ]:
print(pg.mwu(df['por_pos'], df['N']))
print(pg.mwu(df['por_neg'], df['N']))

In [ ]:
print(pg.kruskal(data=df, dv='pos', between='O'))
print(pg.kruskal(data=df, dv='pos', between='O_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='O'))
print(pg.kruskal(data=df, dv='por_pos', between='O_Coded'))
print(pg.kruskal(data=df, dv='neg', between='O'))
print(pg.kruskal(data=df, dv='neg', between='O_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='O'))
print(pg.kruskal(data=df, dv='por_neg', between='O_Coded'))

In [ ]:
print(pg.kruskal(data=df, dv='pos', between='C'))
print(pg.kruskal(data=df, dv='pos', between='C_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='C'))
print(pg.kruskal(data=df, dv='por_pos', between='C_Coded'))
print(pg.kruskal(data=df, dv='neg', between='C'))
print(pg.kruskal(data=df, dv='neg', between='C_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='C'))
print(pg.kruskal(data=df, dv='por_neg', between='C_Coded'))

In [ ]:
print(pg.kruskal(data=df, dv='pos', between='E'))
print(pg.kruskal(data=df, dv='pos', between='E_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='E'))
print(pg.kruskal(data=df, dv='por_pos', between='E_Coded'))
print(pg.kruskal(data=df, dv='neg', between='E'))
print(pg.kruskal(data=df, dv='neg', between='E_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='E'))
print(pg.kruskal(data=df, dv='por_neg', between='E_Coded'))

In [ ]:
print(pg.kruskal(data=df, dv='pos', between='A'))
print(pg.kruskal(data=df, dv='pos', between='A_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='A'))
print(pg.kruskal(data=df, dv='por_pos', between='A_Coded'))
print(pg.kruskal(data=df, dv='neg', between='A'))
print(pg.kruskal(data=df, dv='neg', between='A_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='A'))
print(pg.kruskal(data=df, dv='por_neg', between='A_Coded'))

In [ ]:
print(pg.kruskal(data=df, dv='pos', between='N'))
print(pg.kruskal(data=df, dv='pos', between='N_Coded'))
print(pg.kruskal(data=df, dv='por_pos', between='N'))
print(pg.kruskal(data=df, dv='por_pos', between='N_Coded'))
print(pg.kruskal(data=df, dv='neg', between='N'))
print(pg.kruskal(data=df, dv='neg', between='N_Coded'))
print(pg.kruskal(data=df, dv='por_neg', between='N'))
print(pg.kruskal(data=df, dv='por_neg', between='N_Coded'))

In [ ]:
model = ols('por_pos ~ O + C + E + A + N', data = df).fit()
model.summary()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x1 = df["C"]
x2 = df["A"]
y = df['por_pos']
ax.scatter(x1, x2, y, marker='o')
# Set axis labels
ax.set_xlabel('C')
ax.set_ylabel('A')
ax.set_zlabel('% Positive')

In [ ]:
model = ols('por_neg ~ O + C + E + A + N', data = df).fit()
model.summary()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x1 = df["C"]
x2 = df["E"]
y = df['por_neg']
ax.scatter(x1, x2, y, marker='o')
# Set axis labels
ax.set_xlabel('C')
ax.set_ylabel('E')
ax.set_zlabel('% Negative')

In [ ]:
model = ols('por_pos ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_pos ~ O_Coded + C_Coded + E_Coded + A_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_pos ~ C_Coded + E_Coded + A_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_pos ~ C_Coded + A_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_pos ~ C_Coded', data = df).fit()
model.summary()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x1 = df["C_Coded"]
x2 = df["A_Coded"]
y = df['por_pos']
ax.scatter(x1, x2, y, c='r', marker='o')
# Set axis labels
ax.set_xlabel('C')
ax.set_ylabel('A')
ax.set_zlabel('% Positive')

In [ ]:
model = ols('por_neg ~ O_Coded + C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_neg ~ C_Coded + E_Coded + A_Coded + N_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_neg ~ C_Coded + E_Coded + A_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_neg ~ C_Coded + A_Coded', data = df).fit()
model.summary()

In [ ]:
model = ols('por_neg ~ C_Coded', data = df).fit()
model.summary()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x1 = df["C_Coded"]
x2 = df["E_Coded"]
y = df['por_neg']
ax.scatter(x1, x2, y, c='r', marker='o')
# Set axis labels
ax.set_xlabel('C')
ax.set_ylabel('E')
ax.set_zlabel('% Negative')

In [ ]:
feature_cols = ['O', 'C', 'E', 'A', 'N']
x = df[feature_cols]
y = df['por_pos']

# follow the usual sklearn pattern: import, instantiate, fit

lm = LinearRegression()
lm.fit(x, y)

# print intercept and coefficients
print(lm.intercept_)
print(lm.coef_)

lm.score(x, y)

In [ ]:
from scipy.stats import norm, skew

sns.distplot(df['por_pos'], fit = norm, kde = True, color = 'blue')
plt.plot()

print('The skew of this distribution is = ', skew(df['por_pos']))

In [ ]:
from scipy.stats import norm, skew

sns.distplot(df['O_Coded'], fit = norm, kde = True, color = 'blue')
plt.plot()

print('The skew of this distribution is = ', skew(df['O_Coded']))

In [ ]:
from scipy.stats import norm, skew

sns.distplot(df['C_Coded'], fit = norm, kde = True, color = 'blue')
plt.plot()

print('The skew of this distribution is = ', skew(df['C_Coded']))

In [ ]:
from scipy.stats import norm, skew

sns.distplot(df['E_Coded'], fit = norm, kde = True, color = 'blue')
plt.plot()

print('The skew of this distribution is = ', skew(df['E_Coded']))

In [ ]:
from scipy.stats import norm, skew

sns.distplot(df['A_Coded'], fit = norm, kde = True, color = 'blue')
plt.plot()

print('The skew of this distribution is = ', skew(df['A_Coded']))

In [ ]:
from scipy.stats import norm, skew

sns.distplot(df['N_Coded'], fit = norm, kde = True, color = 'blue')
plt.plot()

print('The skew of this distribution is = ', skew(df['N_Coded']))

In [ ]:
print('The skew of this distribution is = ', skew(df['O_Coded']))
print('The skew of this distribution is = ', skew(df['C_Coded']))
print('The skew of this distribution is = ', skew(df['E_Coded']))
print('The skew of this distribution is = ', skew(df['A_Coded']))
print('The skew of this distribution is = ', skew(df['N_Coded']))

In [ ]:
# df['O_Coded'], lmbda = boxcox(df['O_Coded'])
# df['C_Coded'], lmbda = boxcox(df['C_Coded'])
# df['E_Coded'], lmbda = boxcox(df['E_Coded'])
# df['A_Coded'], lmbda = boxcox(df['A_Coded'])
# df['N_Coded'], lmbda = boxcox(df['N_Coded'])

In [ ]:
print('The skew of this distribution is = ', skew(df['O_Coded']))
print('The skew of this distribution is = ', skew(df['C_Coded']))
print('The skew of this distribution is = ', skew(df['E_Coded']))
print('The skew of this distribution is = ', skew(df['A_Coded']))
print('The skew of this distribution is = ', skew(df['N_Coded']))

In [ ]:
df

In [ ]:
print('The skew of this distribution is = ', skew(df['O_Coded']))
print('The skew of this distribution is = ', skew(df['C_Coded']))
print('The skew of this distribution is = ', skew(df['E_Coded']))
print('The skew of this distribution is = ', skew(df['A_Coded']))
print('The skew of this distribution is = ', skew(df['N_Coded']))

In [ ]:
pg.normality(df)

In [ ]:
model = ols('por_pos ~ C_Coded + A_Coded', data = df).fit()
model.summary()

In [ ]:
from statsmodels.stats.stattools import durbin_watson

#perform Durbin-Watson test
durbin_watson(model.resid)

# Durbin-Watson test: This is done by comparing the successive error terms to check if they are 
# directly/inversely correlated with each other. Values between 1.5–2.5 would tell us that 
# autocorrelation is not a problem in that predictive model.

# 0–1.5 in the Durbin-Watson test refers to a significant positive correlation while 
# 2.5+ refers to a significantly negative correlation.

In [ ]:
feature_cols = ['O', 'C', 'E', 'A', 'N']
X = df[feature_cols]
y = df['por_pos']

X = sm.add_constant(X)

model = sm.OLS(y, X)

results = model.fit()

results.summary()

In [ ]:
feature_cols = ['C', 'E']
X = df[feature_cols]
y = df['por_pos']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print("Intercept : ", regressor.intercept_)
print("COEF: ", regressor.coef_)
print("Score: ", regressor.score(X_train, y_train))
print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))

X_train = sm.add_constant(X_train)
model1 = sm.OLS(y_train,X_train)

result=model1.fit()

print(result.summary())

In [ ]:
feature_cols = ['C', 'A']
X = df[feature_cols]
y = df['por_neg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print("Intercept : ", regressor.intercept_)
print("COEF: ", regressor.coef_)
print("Score: ", regressor.score(X_train, y_train))
print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))

X_train = sm.add_constant(X_train)
model1 = sm.OLS(y_train,X_train)

result=model1.fit()

print(result.summary())

In [ ]:
feature_cols = ['C_Coded', 'A_Coded']
X = df[feature_cols]
y = df['por_pos']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print("Intercept : ", regressor.intercept_)
print("COEF: ", regressor.coef_)
print("Score: ", regressor.score(X_train, y_train))
print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))

X_train = sm.add_constant(X_train)
model1 = sm.OLS(y_train,X_train)

result=model1.fit()

print(result.summary())

In [ ]:
feature_cols = ['C_Coded', 'A_Coded']
X = df[feature_cols]
y = df['por_neg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print("Intercept : ", regressor.intercept_)
print("COEF: ", regressor.coef_)
print("Score: ", regressor.score(X_train, y_train))
print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))

X_train = sm.add_constant(X_train)
model1 = sm.OLS(y_train,X_train)

result=model1.fit()

print(result.summary())

In [ ]:
feature_cols = ['C_Coded', 'A_Coded']
X = df[feature_cols]
y = df['por_neg']

results = []

for n in range(1, 11):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=n)

    kf = KFold(n_splits=5, shuffle=True)

    lm = LinearRegression()
    # scores = cross_val_score(lm, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    scores = cross_val_score(lm, X_train, y_train, scoring="r2", cv=kf)
#     print(scores)
    # print(sorted(me.SCORERS.keys()))
    # print(np.absolute(np.mean(scores)))
    predictions = cross_val_predict(lm, X_train, y_train, cv=kf)
#     print(predictions)
    accuracy = me.r2_score(y_train, predictions)
    print("Cross-Predicted Accuracy: ", accuracy)
    results.append(accuracy)
    
print(np.mean(results))

In [ ]:
predictions = cross_val_predict(lm, X_train, y_train, cv=5)
predictions

In [ ]:
accuracy = me.r2_score(y_train, predictions)
print("Cross-Predicted Accuracy: ", accuracy)

In [ ]:
X.loc[3]

In [ ]:
feature_cols = ['C_Coded', 'A_Coded']
X = df[feature_cols]
y = df['por_pos']

kf = KFold(n_splits=5, random_state=1, shuffle=True)

scores = []

for train_index, test_index in kf.split(X):
#         print("Train:", train_index, "Validation:",test_index)
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)

        y_pred = regressor.predict(X_test)

#         print("Intercept : ", regressor.intercept_)
#         print("COEF: ", regressor.coef_)
#         print("Score: ", regressor.score(X_train, y_train))
#         print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
#         print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
#         print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))
        scores.append(regressor.score(X_train, y_train))
    
print(np.mean(scores))

In [ ]:
feature_cols = ['C_Coded', 'A_Coded']
X = df[feature_cols]
y = df['por_pos']

# evaluate a model with a given number of repeats
def evaluate_model(X_train, y_train, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=5, n_repeats=repeats, random_state=1)
    # create model
    model = LinearRegression()
    # evaluate model
    scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=cv, n_jobs=-1)
    print(scores)
    return scores

# configurations to test
repeats = range(1,6)
results = list()
for r in repeats:
    # evaluate using a given number of repeats
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    scores = evaluate_model(X_train, y_train, r)
    # summarize
#     print('>%d mean=%.4f se=%.3f' % (r, np.mean(scores), stats.sem(scores)))
    # store
    results.append(np.mean(scores))
    
print(np.mean(results))